In [ ]:
import pandas as pd
from pathlib import Path
import pymc as pm
import plotly.express as px
import numpy as np
import pandas as pd

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm
from tbdynamics import model
from tbdynamics.utils import round_sigfig
from tbdynamics.inputs import load_params
from tbdynamics.constants import age_strata, organ_strata, compartments, latent_compartments, infectious_compartments
from tbdynamics.plotting import plot_model_vs_actual

## Define variables

In [ ]:
PROJECT_PATH = Path().resolve()
DATA_PATH = PROJECT_PATH / 'data'
pd.options.plotting.backend = "plotly"

time_start = 1700.0
time_end = 2023.0
time_step = 0.1
matrix = np.ones((6, 6)) # Homo mixing


In [ ]:
fixed_parameters = load_params(PROJECT_PATH / 'tbdynamics/params.yml')

## Define Model

### Base model

In [ ]:
tb_model = model.build_model(
    compartments,
    latent_compartments,
    infectious_compartments,
    age_strata,
    time_start,
    time_end,
    time_step,
    fixed_parameters,
    matrix
)

### Params and calibration targets

In [ ]:
params = {}


priors = [
    esp.UniformPrior("start_population_size", (100000, 30000000)),
    esp.UniformPrior("contact_rate", (0.1, 130)),  # multiplied with eigenvalue
    esp.UniformPrior("rr_infection_latent", (0.2, 0.5)),
    esp.UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    esp.UniformPrior("progression_multiplier", (0.1, 10.0)),
    esp.UniformPrior("seed_time", [1700.0, 1900.0]),
    esp.UniformPrior("seed_duration", [1.0, 5.0]),
    esp.UniformPrior("seed_num", [1.0, 1500.0]),
    esp.UniformPrior("smear_positive_death_rate", (0.335, 0.449)),
    esp.UniformPrior("smear_negative_death_rate", (0.017, 0.035)),
    esp.UniformPrior("smear_positive_self_recovery", (0.177, 0.288)),
    esp.UniformPrior("smear_negative_self_recovery", (0.073, 0.209)),
]

pop = pd.Series(
    {
        2009: 86025000,
        2019: 96484000,
    }
)

notifs = pd.Series(
    {
        2010: 99022,
        2011: 100518,
        2012: 103906,
        2013: 102196,
        2014: 102087,
        2015: 102676,
        2016: 102527,
        2017: 105733,
        2018: 102171,
        2019: 104505,
        2020: 101795,
        2021: 78935,
        2022: 103804,
    }
)

prop_smear_positive = pd.Series({2022: .83})
incidence = pd.Series({2022: 176})

targets = [
    est.NormalTarget("total_population", pop, stdev=100000.0),
    est.NormalTarget("notification", notifs, stdev=1000.0),
    est.NormalTarget("prop_smear_positive", prop_smear_positive, stdev=.5),
    est.NormalTarget("incidence", incidence, stdev=33.0),
]

calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

### Running Calibration

In [ ]:
with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *calibration_model.priors[k].bounds(0.99)) for k, v in params.items() if k in calibration_model.priors}
    variables = epm.use_model(calibration_model)
    map_params = pm.find_MAP(start=start_params, vars=variables, maxeval= 10000,include_transformed=False)
    map_params = {k: float(v) for k, v in map_params.items()}
    print('Best calibration parameters found:')
for i_param, param in enumerate(map_params):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')

In [ ]:
params.update(map_params)
tb_model.run(params)
derived_df_0 = tb_model.get_derived_outputs_df()

### Population output

In [ ]:
plot_model_vs_actual(derived_df_0, pop, 'total_population', 'Population', 'Modelled vs Data')

In [ ]:
derived_df_0[[f'total_populationXage_{i}' for i in age_strata]].plot(title='Modelled populatation by age group', kind ='area')

In [ ]:
derived_df_0['incidence'].plot()

In [ ]:
derived_df_0['prevalence_infectious'].plot()

In [ ]:
derived_df_0[[f"prop_{compartment}" for compartment in compartments]].plot(kind="area")

In [ ]:
derived_df_0[[f'total_populationXorgan_{i}' for i in organ_strata]].plot(title='Modelled populatation by organ status', kind ='area')

In [ ]:
derived_df_0[[f"prop_{organ_stratum}" for organ_stratum in organ_strata]].plot(kind="area")

In [ ]:
plot_model_vs_actual(derived_df_0, notifs, 'notification', 'Notification', 'Modelled vs Data')